In [1]:
import os 
import sys 
import openai
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.chdir(r"C:\Users\TempAccess\Documents\Dhruv\RAG")
print(os.getcwd())

C:\Users\TempAccess\Documents\Dhruv\RAG


In [3]:
from helper_function_openai import (
    SimpleRAG,
    RAGRetriever,
    OpenAIEmbedder,
    OpenAIChat,
    FAISSVectorStore,
    chunk_text,
    chunk_documents,
    show_context,
    Document,
    read_csv_as_documents
)

In [4]:
import pandas as pd 
import numpy as np 

In [5]:
!pip install fsspec 


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\TempAccess\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [6]:
df = pd.read_csv(r"C:\Users\TempAccess\Documents\Dhruv\RAG\data\customers-100.csv")
df.head(5)

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


In [7]:
df.shape

(100, 12)

In [8]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Index              100 non-null    int64
 1   Customer Id        100 non-null    str  
 2   First Name         100 non-null    str  
 3   Last Name          100 non-null    str  
 4   Company            100 non-null    str  
 5   City               100 non-null    str  
 6   Country            100 non-null    str  
 7   Phone 1            100 non-null    str  
 8   Phone 2            100 non-null    str  
 9   Email              100 non-null    str  
 10  Subscription Date  100 non-null    str  
 11  Website            100 non-null    str  
dtypes: int64(1), str(11)
memory usage: 9.5 KB


In [9]:
df.columns

Index(['Index', 'Customer Id', 'First Name', 'Last Name', 'Company', 'City',
       'Country', 'Phone 1', 'Phone 2', 'Email', 'Subscription Date',
       'Website'],
      dtype='str')

In [10]:
df.columns.tolist()

['Index',
 'Customer Id',
 'First Name',
 'Last Name',
 'Company',
 'City',
 'Country',
 'Phone 1',
 'Phone 2',
 'Email',
 'Subscription Date',
 'Website']

In [11]:
for group_key, group_df  in df.groupby("First Name"):
    print(f"{group_key}\n{group_df}")
    break

Aimee
   Index      Customer Id First Name Last Name       Company           City  \
5      6  2d08FB17EE273F4      Aimee     Downs  Steele Group  Chavezborough   

                  Country              Phone 1       Phone 2  \
5  Bosnia and Herzegovina  (283)437-3886x88321  999-728-1637   

                 Email Subscription Date                 Website  
5  louis27@gilbert.com        2020-02-25  http://www.berger.net/  


In [12]:
df.shape

(100, 12)

## Lets now convert CSV to Document.


In [13]:
documents = read_csv_as_documents(file_path=r"C:\Users\TempAccess\Documents\Dhruv\RAG\data\customers-100.csv")

documents

[Document(content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/', metadata={'row_index': 0, 'source': 'C:\\Users\\TempAccess\\Documents\\Dhruv\\RAG\\data\\customers-100.csv', 'Index': '1', 'Customer Id': 'DD37Cf93aecA6Dc', 'First Name': 'Sheryl', 'Last Name': 'Baxter', 'Company': 'Rasmussen Group', 'City': 'East Leonard', 'Country': 'Chile', 'Phone 1': '229.077.5154', 'Phone 2': '397.884.0519x718', 'Email': 'zunigavanessa@smith.info', 'Subscription Date': '2020-08-24', 'Website': 'http://www.stephenson.com/'}, embedding=None),
 Document(content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: v

In [14]:
def csv_to_documents(df: pd.DataFrame, text_columns:list= None) -> list:
    """
    Convert DataFrame rows into Document objects for RAG indexing.
    
    Each row becomes a Document with:
    - content: structured text representation of all columns
    - metadata: row index + individual column values for filtering
    
    Args:
        df: Input DataFrame
        text_columns: Optional subset of columns to include in content.
                      If None, all columns are used.
    Returns:
        List of Document objects
    """

    documents = []

    columns = text_columns or df.columns.tolist()

    for idx, row in df.iterrows():
        text_parts = []

        for col in columns:
            value = row[col]

            if pd.notna(value):
                text_parts.append(f"{col}:{value}")

        content = "\n".join(text_parts)

        metadata = {
            "row_index": int(idx),
            "source": "customers_100.csv"
        }

        for col in columns:
            if pd.notna(row[col]):
                metadata[col] = str(row[col])

        documents.append(Document(content=content, metadata=metadata))

    return documents

In [15]:
documents = csv_to_documents(df)
documents

[Document(content='Index:1\nCustomer Id:DD37Cf93aecA6Dc\nFirst Name:Sheryl\nLast Name:Baxter\nCompany:Rasmussen Group\nCity:East Leonard\nCountry:Chile\nPhone 1:229.077.5154\nPhone 2:397.884.0519x718\nEmail:zunigavanessa@smith.info\nSubscription Date:2020-08-24\nWebsite:http://www.stephenson.com/', metadata={'row_index': 0, 'source': 'customers_100.csv', 'Index': '1', 'Customer Id': 'DD37Cf93aecA6Dc', 'First Name': 'Sheryl', 'Last Name': 'Baxter', 'Company': 'Rasmussen Group', 'City': 'East Leonard', 'Country': 'Chile', 'Phone 1': '229.077.5154', 'Phone 2': '397.884.0519x718', 'Email': 'zunigavanessa@smith.info', 'Subscription Date': '2020-08-24', 'Website': 'http://www.stephenson.com/'}, embedding=None),
 Document(content='Index:2\nCustomer Id:1Ef7b82A4CAAD10\nFirst Name:Preston\nLast Name:Lozano\nCompany:Vega-Gentry\nCity:East Jimmychester\nCountry:Djibouti\nPhone 1:5153435776\nPhone 2:686-620-1820x944\nEmail:vmata@colon.com\nSubscription Date:2021-04-23\nWebsite:http://www.hobbs.com

## now lets Build vector store.

In [16]:
embedder = OpenAIEmbedder(model="text-embedding-3-small")

documents = embedder.embed_documents(documents=documents)

documents

[Document(content='Index:1\nCustomer Id:DD37Cf93aecA6Dc\nFirst Name:Sheryl\nLast Name:Baxter\nCompany:Rasmussen Group\nCity:East Leonard\nCountry:Chile\nPhone 1:229.077.5154\nPhone 2:397.884.0519x718\nEmail:zunigavanessa@smith.info\nSubscription Date:2020-08-24\nWebsite:http://www.stephenson.com/', metadata={'row_index': 0, 'source': 'customers_100.csv', 'Index': '1', 'Customer Id': 'DD37Cf93aecA6Dc', 'First Name': 'Sheryl', 'Last Name': 'Baxter', 'Company': 'Rasmussen Group', 'City': 'East Leonard', 'Country': 'Chile', 'Phone 1': '229.077.5154', 'Phone 2': '397.884.0519x718', 'Email': 'zunigavanessa@smith.info', 'Subscription Date': '2020-08-24', 'Website': 'http://www.stephenson.com/'}, embedding=[0.04553413763642311, -0.011052267625927925, 0.016849437728524208, 0.03773430734872818, -0.006576400250196457, -0.004558683838695288, -0.007581494282931089, 0.06059172376990318, -0.023263968527317047, -0.06775913387537003, 0.009147482924163342, -0.05107532814145088, 0.01754208654165268, -0.0

In [17]:
print(f"Embedding {len(documents)} Documents")
print(f"Embedding dimension: {embedder.dimension}")

Embedding 100 Documents
Embedding dimension: 1536


In [18]:
vector_store = FAISSVectorStore(dimension=embedder.dimension)
vector_store

In [19]:
vector_store.add_documents(documents=documents)

## its query time

In [20]:
chat = OpenAIChat(
    model_name="gpt-4o-mini",
    temperature=0.0,
    max_tokens=500
)

chat

In [21]:
def query_csv(question: str, k:int = 4) -> str:
    """
    End-to-End CSV RAG query.

    1. Embed the question
    2. Search FAISS for top-k similar rows
    3. Pass context to OpenAI chat for answer generation
    """
    
    # 1. Embed the question
    question_embedding = embedder.embed_text(question)
    # print(f"Embed question: {question_embedding}")

    # print(f"Question: {question}")
    results = vector_store.search(query_embeddings=question_embedding, k=k)
    # print(f"Context that i got: {results}")

    # Context string 
    context = [r.document.content for r in results]
        
    # print(f"Final context: {context}")
    system_prompt = (
        "You are a customer data assistant."
        "Use the following customer records to answer the question."
        "If the answer isn't in the data, say so. "
        "Be concise and precise."
    )

    answer = chat.chat_with_context(
        query = question,
        context= context,
        system_prompt= system_prompt
    )

    return answer

In [22]:
question = "Which company does Sheryl Baxter work for?"
answer = query_csv(question)

print(f"Q: {question}")
print(f"A: {answer}")

Q: Which company does Sheryl Baxter work for?
A: Sheryl Baxter works for Rasmussen Group.


In [23]:
save_path = "./data/customer_data_vector_store"

vector_store.save(save_path)


## Alternative approach 
## Directly use Simple RAG class

In [24]:
rag = SimpleRAG(
    embedding_model="text-embedding-3-small",
    chat_model="gpt-3.5-turbo",
    temperature=0.3
)

rag.index_csv(r"data/customers-100.csv")

100

In [25]:
result = rag.query("Which company does Sheryl Baxter work for?")
print(f"Answer: {result['answer']}")

Answer: Sheryl Baxter works for the Rasmussen Group.


In [26]:
rag.index_pdf(r"data/Understanding_Climate_Change.pdf")

97

In [27]:
question = "What is the main cause of climate change?"
result = rag.query(question, return_context=True)

In [28]:
print(f"Answer: {result['answer']}")

Answer: The main cause of climate change is the increase in greenhouse gases in the atmosphere, particularly carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), which trap heat from the sun and create a "greenhouse effect."


In [29]:
rag.index_text(f"""Hello There i am Dhruv Panchal, i am an software engineer working at Capgemini Technology Services India Limited, i am from Ahmedabad, Gujarat, i have completed my Bachelor of Engineering in Computer Science and Engineering from Gujarat Technological University, i have 2 years of experience in software development, i am a quick learner and a team player, i am looking for new opportunities to grow my career, i am passionate about technology and innovation, i am a team player and a quick learner, i am looking for new opportunities to grow my career""")

1

In [30]:
question = "What is my name and where do i work?? also for general knowledge i want to ask what is the main reason for climate change?"
result = rag.query(question, k= 4, return_context=True)
print(f"Answer: {result['answer']}")
print(f"\nContext: {result['context']}")

Answer: Based on the context provided, I cannot determine your name or where you work. The main reason for climate change, as stated in the text, is the increase in greenhouse gases in the atmosphere, particularly carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), which trap heat from the sun and create a "greenhouse effect."

Context: ['man civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which began at the end of the last ice age, human societies flourished, but the industrial era has seen \nunprecedented changes. \nModern Observations \nModern scientific observations indicate a rapid increase in global temperatures, sea levels, \nand extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has \ndocumented these changes extensively. Ice core samples, tree rings, and ocean sediments \nprovide a historical record

In [31]:
print(f"\nContext: {len(result['context'])}")


Context: 4


In [32]:
print(f"\nContext: {result['context'][0]}")


Context: man civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which began at the end of the last ice age, human societies flourished, but the industrial era has seen 
unprecedented changes. 
Modern Observations 
Modern scientific observations indicate a rapid increase in global temperatures, sea levels, 
and extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has 
documented these changes extensively. Ice core samples, tree rings, and ocean sediments 
provide a historical record that scientists use to understand past climate conditions and 
predict future trends. The evidence overwhelmingly shows that recent changes are primarily 
driven by human activities, particularly the emission of greenhouse gases. 
Chapter 2: Causes of Climate Change 
Greenhouse Gases 
The primary cause of recent climate change is the increase in g

In [33]:
print(f"\nContext: {result['context'][1]}")


Context: Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change. 
Historical Context 
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and human civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which


In [34]:
print(f"\nContext: {result['context'][2]}")


Context: e Education in Schools 
Integrating climate education into school curricula is essential for preparing future 
generations to address climate challenges. Climate education should cover the science of climate change, its impacts, and solutions. Experiential learning, such as outdoor education 
and hands-on projects, enhances understanding and engagement. 
Public Awareness Campaigns 
Public awareness campaigns aim to inform and educate the general public about climate 
change and encourage sustainable behaviors. Effective campaigns use clear messaging, 
relatable examples, and actionable tips. Collaboration with media, influencers, and 
community organizations enhances reach and impact. 
Lifelong Learning 
Lifelong learning initiatives provide opportunities for people of all ages to learn about 
climate change and sustainability. This includes adult education programs, online courses, 
workshops, and community events. Lifelong learning fosters a culture of continuous 
improveme

In [35]:
print(f"\nContext: {result['context'][3]}")


Context: d system. 
By understanding the causes, effects, and potential solutions to climate change, we can take 
informed actions to protect our planet for future generations. Global cooperation, innovation, and commitment are key to addressing this pressing challenge. 
 
Chapter 5: The Role of Technology in Climate Change 
Mitigation 
Advanced Renewable Energy Solutions 
Next-Generation Solar Technologies


In [37]:
print(f"\nContext: {result['context'][4]}")

IndexError: list index out of range

In [44]:
question = "what is the main reason for climate change?. Where Dhruv is from??"
result = rag.query(question, k= 10, return_context=True)

print(f"Answer: {result['answer']}")
print(f"\nContext: {result['context']}")

Answer: The main reason for climate change is the increase in greenhouse gases in the atmosphere, primarily due to human activities such as burning fossil fuels and deforestation. Dhruv is from Ahmedabad, Gujarat, India.

Context: ['man civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the amount of solar energy our planet receives. During the Holocene epoch, which began at the end of the last ice age, human societies flourished, but the industrial era has seen \nunprecedented changes. \nModern Observations \nModern scientific observations indicate a rapid increase in global temperatures, sea levels, \nand extreme weather events. The Intergovernmental Panel on Climate Change (IPCC) has \ndocumented these changes extensively. Ice core samples, tree rings, and ocean sediments \nprovide a historical record that scientists use to understand past climate conditions and \npredict future trends. The evidence overwhelmingly sh

In [45]:
print(f"\nContext: {len(result['context'])}")


Context: 10


In [46]:
rag.show_context(question, k=50)

Query: what is the main reason for climate change?. Where Dhruv is from??


Context 1:
----------------------------------------
man civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which began at the end of the last ice age, human societies flourished, but the industrial era has seen 
unprecedented changes. 
Modern Observations 
Modern scientific observations indicate a rapid increase in global temperatures, sea levels, 
and extreme weather events. The Intergovernmental Panel on Cli...


Context 2:
----------------------------------------
Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. O